# 1 Chargement du modèle de prediction

In [2]:
# 1) Imports
import joblib
import pandas as pd
import numpy as np

# 2) Charger le pipeline sauvegardé
bundle = joblib.load("rf_no_pca_pipeline.joblib")
pipe = bundle["pipeline"]
FEATURES = bundle["features"]
print(" Modèle chargé. Features attendues :", FEATURES)


 Modèle chargé. Features attendues : ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']


# 2 Chargement du fichier à prédire

### attention : le fichier d’entrée doit contenir en colonnes au moins diagonal, height_left, height_right, margin_low, margin_up, length (noms exacts)

In [5]:
# Indique le chemin du fichier à prédire (séparateur ; comme dans ma base de données initial )
INPUT_PATH = "billets_test.csv"   # <-- à adapter
OUTPUT_PATH = "billets_predictions.csv" # <-- où je veux écrire le résultat

# Chargement 
df_in = pd.read_csv(INPUT_PATH) #ajouter => , sep=";" si séparateur ;

print("Aperçu du fichier d'entrée :")
display(df_in.head())
print(df_in.shape)


Aperçu du fichier d'entrée :


,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


(5, 7)


# 3 Contrôle et prépartion des valeurs manquantes

In [7]:
# 3.1 Vérifier la présence des colonnes nécessaires
missing_cols = [c for c in FEATURES if c not in df_in.columns]
if missing_cols:
    raise ValueError(f"Colonnes manquantes dans le fichier d'entrée : {missing_cols}")

# 3.2 Conserver uniquement les features nécessaires (on laisse les autres pour les réattacher après)
df_feat = df_in[FEATURES].copy()

# 3.3 Aligner le traitement avec l'entraînement : suppression des lignes incomplètes
before = len(df_feat)
mask_complete = df_feat[FEATURES].notna().all(axis=1)
df_feat = df_feat[mask_complete]
dropped = before - len(df_feat)

# Conserver les mêmes lignes côté df_in pour réattacher les prédictions proprement
df_clean_in = df_in.loc[mask_complete].copy()

print(f" Lignes complètes conservées : {len(df_feat)} (lignes supprimées : {dropped})")


 Lignes complètes conservées : 5 (lignes supprimées : 0)


# 4 Prédire (probailités + classes)

In [9]:
# 4.1 Probabilités de "vrai billet" (classe True) -> colonne 1 si y est booléen
proba_true = pipe.predict_proba(df_feat)[:, 1]

# 4.2 Classe booléenne (seuil 0.5 par défaut)
pred_bool = proba_true >= 0.5

# Seuil de décision : 0.5 par défaut. Si, métier, on veut encore moins de faux négatifs, 
# on peut baisser le seuil (ex. 0.45) pour être plus “strict” sur la détection des faux.

# 4.3 Étiquette lisible
pred_label = np.where(pred_bool, "Vrai billet", "Faux billet")

# 4.4 Réattacher au DataFrame d'entrée filtré
df_out = df_clean_in.copy()
df_out["proba_true"] = proba_true.round(4)
df_out["prediction_bool"] = pred_bool
df_out["prediction_label"] = pred_label

print("Aperçu des prédictions :")
display(df_out.head())


Aperçu des prédictions :


,diagonal,height_left,height_right,margin_low,margin_up,length,id,proba_true,prediction_bool,prediction_label
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,1.0000,True,Vrai billet
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,0.0000,False,Faux billet
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,1.0000,True,Vrai billet
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,0.0033,False,Faux billet
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,0.0033,False,Faux billet


# 5 Export du fichier enrichi

In [11]:
# Sauvegarde (même séparateur ;)
df_out.to_csv(OUTPUT_PATH, index=False) # si séparateur ";" alors ajouter après output_path => , sep=";"
print(f" Fichier prédictions écrit : {OUTPUT_PATH}")
print("Résumé des classes prédites :")
print(df_out["prediction_label"].value_counts())


 Fichier prédictions écrit : billets_predictions.csv
Résumé des classes prédites :
prediction_label
Faux billet    3
Vrai billet    2
Name: count, dtype: int64


# 6 Bonus : si fichier contient quand même is_genuine (pour test interne)

In [13]:
if "is_genuine" in df_out.columns:
    from sklearn.metrics import classification_report, confusion_matrix
    print("\n🔎 Évaluation interne (car 'is_genuine' présent) :")
    print(classification_report(df_out["is_genuine"], df_out["prediction_bool"],
                                target_names=["Faux billet", "Vrai billet"]))
    print("Matrice de confusion :")
    print(confusion_matrix(df_out["is_genuine"], df_out["prediction_bool"]))


# 7 Bonus bis : Zone d'abstention autour de 0.5

In [15]:
# --- Paramètres de triage (à ajuster) ---
LOW  = 0.45   # seuil bas de la zone d’abstention
HIGH = 0.55   # seuil haut de la zone d’abstention

# Probas déjà calculées plus haut : proba_true = pipe.predict_proba(df_feat)[:, 1]

# 1) Catégorie tri-états
def triage_label(p):
    if p < LOW:
        return "Faux billet"
    elif p > HIGH:
        return "Vrai billet"
    else:
        return "À vérifier"   # zone d’abstention

df_out = df_clean_in.copy()
df_out["proba_true"] = proba_true
df_out["prediction_3classes"] = [triage_label(p) for p in df_out["proba_true"]]

# 2) Résumé
print("Répartition des prédictions (triage 3 classes) :")
print(df_out["prediction_3classes"].value_counts(dropna=False))




Répartition des prédictions (triage 3 classes) :
prediction_3classes
Faux billet    3
Vrai billet    2
Name: count, dtype: int64
